In [1]:
!pip install dash


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import yfinance as yf
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from datetime import datetime

app = dash.Dash(__name__)

# Function to fetch stock data
def get_stock_data(value, period):
    return yf.Ticker(value).history(period=period)

# Function to generate stock price chart
def generate_stock_chart(stock_data):
    ma_short = stock_data['Close'].rolling(window=20).mean()
    ma_long = stock_data['Close'].rolling(window=50).mean()

    fig = {
        'data': [
            {'x': stock_data.index, 'y': stock_data['Close'], 'type': 'line', 'name': 'Close', 'line': {'color': '#1f77b4'}},
            {'x': stock_data.index, 'y': ma_short, 'type': 'line', 'name': 'MA (20)', 'line': {'color': '#ff7f0e'}},
            {'x': stock_data.index, 'y': ma_long, 'type': 'line', 'name': 'MA (50)', 'line': {'color': '#2ca02c'}},
            {'x': stock_data.index, 'y': stock_data['Volume'], 'type': 'bar', 'name': 'Volume', 'marker': {'color': '#d62728'}},
        ],
        'layout': {
            'title': 'Stock Price Dashboard',
            'yaxis': {'title': 'Price'},
            'yaxis2': {'title': 'Volume', 'overlaying': 'y', 'side': 'right'},
            'legend': {'orientation': 'h', 'y': 1.1, 'x': 0.5},
            'plot_bgcolor': '#f0f0f0'
        }
    }

    return fig

# Function to generate candlestick chart
def generate_candlestick_chart(stock_data):
    fig = go.Figure(data=[go.Candlestick(x=stock_data.index,
                                          open=stock_data['Open'],
                                          high=stock_data['High'],
                                          low=stock_data['Low'],
                                          close=stock_data['Close'])])
    fig.update_layout(title='Stock Price (Candlestick)',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      xaxis_rangeslider_visible=False,
                      plot_bgcolor='#f0f0f0')
    return fig

# Function to generate summary
def generate_summary(stock_data):
    close_prices = stock_data['Close']
    change = close_prices.iloc[-1] - close_prices.iloc[-2]
    percent_change = abs(change / close_prices.iloc[-2]) * 100
    if change > 0:
        return html.P(f"Stock has moved up by {percent_change:.2f}% compared to yesterday.", style={'color': 'green'})
    elif change < 0:
        return html.P(f"Stock has moved down by {percent_change:.2f}% compared to yesterday.", style={'color': 'red'})
    else:
        return html.P("No significant change compared to yesterday.", style={'color': 'yellow'})


# Function to generate overall summary
def generate_overall_summary(stocks_data):
    summary_texts = ['Overview of all Stocks: ']
    for stock_data, stock_symbol in zip(stocks_data, ['AAPL', 'AMZN', 'MSFT', 'GOOGL', 'TSLA']):
        close_prices = stock_data['Close']
        change = close_prices.iloc[-1] - close_prices.iloc[-2]
        if change > 0:
            summary_texts.append(f"{stock_symbol}: Up.")
        elif change < 0:
            summary_texts.append(f"{stock_symbol}: Down.")
        else:
            summary_texts.append(f"{stock_symbol}: No significant change.")
    return html.P("  ".join(summary_texts), style={'font-size': 'small', 'color': 'yellow'})


# App layout
app.layout = html.Div(children=[
    html.H1(children=['Stock Price Dashboard', generate_overall_summary([get_stock_data(stock, '2d') for stock in ['AAPL', 'AMZN', 'MSFT', 'GOOGL', 'TSLA']])], style={'display': 'flex', 'justify-content': 'space-between', 'color': '#1f77b4'}),
    
    dcc.Dropdown(
        id='dropdown',
        options=[
            {'label': 'Apple', 'value': 'AAPL'},
            {'label': 'Amazon', 'value': 'AMZN'},
            {'label': 'Microsoft', 'value': 'MSFT'},
            {'label': 'Google', 'value': 'GOOGL'},
            {'label': 'Tesla', 'value': 'TSLA'}
        ],
        value='AAPL',
        style={'width': '50%', 'margin': 'auto', 'margin-bottom': '30px'}
    ),

    dcc.Slider(
        id='time-slider',
        min=1,
        max=40 * 365,
        step=1,
        value=40 * 365,  # Default to last year
        marks={i * 365: datetime.now().year - i for i in range(40, 0, -1)},  # Marks for each year
        tooltip={'placement': 'bottom'}
    ),

    html.Div([
        dcc.Graph(id='stock-chart', style={'width': '650px', 'height': '400px'}),
        dcc.Graph(id='candlestick-chart', style={'width': '650px', 'height': '400px'})
    ], style={'display': 'flex', 'flex-direction': 'row', 'justify-content': 'space-between'}),

    # Summary of stock movement
    html.Div(id='stock-summary', style={'margin-top': '20px', 'text-align': 'center'})
])

# Callback to update stock chart and summary
@app.callback(
    [Output('stock-chart', 'figure'), Output('stock-summary', 'children')],
    [Input('dropdown', 'value'), Input('time-slider', 'value')]
)
def update_stock_chart(value, time_period):
    stock_data = get_stock_data(value, f'{time_period}d')
    fig = generate_stock_chart(stock_data)
    summary = generate_summary(stock_data)
    return fig, summary

# Callback to update candlestick chart
@app.callback(
    Output('candlestick-chart', 'figure'),
    Input('dropdown', 'value'),
    Input('time-slider', 'value')
)
def update_candlestick_chart(value, time_period):
    stock_data = get_stock_data(value, f'{time_period}d')
    fig = generate_candlestick_chart(stock_data)
    return fig

if __name__ == '__main__':
    app.run_server()
